#### Load data

In [36]:
with open('log/IIIHTTP2.log','r') as InputData:
    APPdata = InputData.readlines()
    APPdata = [i.strip().split('\t') for i in APPdata]
    APPdata = list(filter(lambda x:'#' not in x[0],APPdata))

In [37]:
with open('log/IIIHTTP3.log','r') as InputData2:
    Testdata = InputData2.readlines()
    Testdata = [i.strip().split('\t') for i in Testdata]
    Testdata = list(filter(lambda x:'#' not in x[0],Testdata))

In [38]:
def similarity(test_data,whiteList_data):
    score = 0
    for key in fingerprint_keys_index:
        for key_index in fingerprint_keys_index[key]:
            if test_data[key_index] in whiteList_data[key]:
                score += 1
    #if score > 0:
    #    WhiteListUpdate(test_data,whiteList_data)
    return score

In [39]:
def WhiteListUpdate(data):
    pass

In [40]:
def outgoingInfo(label,data):
    req_len = data[13]
    res_len = data[14]
    #label = data[12]
    if req_len > trainedList[label]['request_len']:
        print(req_len,trainedList[label]['request_len'])
        #return 
    elif res_len > trainedList[label]['respond_len']:
        print(res_len,trainedList[label]['respond_len'])
    else:
        return 

In [41]:
def average_data_len(label,req_data,res_data,num):
    average_req = 0
    average_res = 0
    if req_data != 0:
        num[0] += 1
        average_req = (float(trainedList[label]['request_len']) *(num[0] - 1) + req_data)/num[0]
    if res_data != 0:
        num[1] += 1
        average_res = (float(trainedList[label]['respond_len']) *(num[1] - 1) + req_data)/num[1]
    return average_req,average_res,num

#### Training the white list fingerprint

In [47]:
fingerprint_keys_index={'ip':[4],'host':[8],'userAgent':[12]}
trainedList = {}
for index in range(len(APPdata)):
    if APPdata[index][12].strip() != "-":
        label = APPdata[index][12].replace('\'','').strip()
        if label not in trainedList:
            tmp_dict = {'ip':[APPdata[index][4].replace('\'','').strip()],
                        'host':[APPdata[index][8].replace('\'','').strip()],
                        'userAgent':[label],
                       'num':[0,0]}
            trainedList.setdefault(label)
            trainedList[label] = tmp_dict
            if int(APPdata[index][13].replace('\'','').strip()) != 0:
                trainedList[label]['request_len'] = float(APPdata[index][13].replace('\'','').strip())
            else:
                trainedList[label]['request_len'] = []
            if int(APPdata[index][14].replace('\'','').strip()) != 0:
                trainedList[label]['respond_len'] = float(APPdata[index][14].replace('\'','').strip())
            else:
                trainedList[label]['respond_len'] = []
        else:
            for key in fingerprint_keys_index:
                for key_index in fingerprint_keys_index[key]:
                    tmp_APPdata = APPdata[index][key_index].replace('\'','').strip()
                    if  tmp_APPdata not in trainedList[label][key]:
                        trainedList[label].setdefault(key,[]).append(tmp_APPdata) 
        trainedList[label]['request_len'],trainedList[label]['respond_len'],trainedList[label]['num'] = average_data_len(label,float(APPdata[index][13]),float(APPdata[index][14]),trainedList[label]['num'])

In [54]:
print(trainedList['Mozilla/5.0 (Windows NT 6.1; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/65.0.3325.162 Safari/537.36'])

{'ip': ['210.69.151.47', '216.58.200.34', '104.116.1.153', '172.217.160.74', '172.217.160.66', '103.11.102.36', '74.125.204.82', '152.195.37.240', '172.217.160.106', '103.23.108.184', '104.124.226.79', '103.23.108.140', '152.195.38.13', '54.192.212.210', '103.23.108.114', '52.94.234.174', '54.248.119.10', '13.32.56.69', '130.211.5.208', '103.23.108.122', '172.217.160.72', '103.23.108.110', '103.23.108.180', '103.23.108.181', '103.23.108.119', '185.206.85.82', '185.206.86.13', '185.206.85.85', '173.254.212.119', '52.85.200.100', '173.254.109.169', '163.29.3.98', '166.62.6.39', '192.0.77.2', '211.20.181.199', '104.124.242.18', '104.124.235.63', '75.126.52.130', '104.239.130.82', '151.101.88.188', '216.58.200.46', '172.217.27.142', '108.177.97.104', '151.101.230.110', '54.192.212.73', '35.190.23.218', '104.20.55.219', '172.217.27.130', '61.221.230.163', '203.69.138.91', '107.178.241.176', '52.222.149.59', '210.176.156.31', '168.63.208.90', '182.161.72.73', '210.176.156.43', '104.155.231.1

In [34]:
toBeDetermined = []
NoHost =[]
for test_index in range(len(Testdata)):
    similarityScore = 0
    if Testdata[test_index][8] == '-':
        NoHost.append(Testdata[test_index])
    else:
        if Testdata[test_index][12] in trainedList:
            whiteList = trainedList[Testdata[test_index][12]]
            similarityScore = similarity(Testdata[test_index],whiteList)
        else:
            max_score = 0
            likelihoodAPP = []
            for label in trainedList:
                similarityScore = similarity(Testdata[test_index],trainedList[label])
                if similarityScore > max_score:
                    likelihoodAPP = Testdata[test_index]
                    max_score = similarityScore
                elif (similarityScore > 0) and (similarityScore < max_score):
                    likelihoodAPP.append(Testdata[test_index])
            if max_score == 0:
                toBeDetermined.append(Testdata[test_index])
            #print(similarityScore,Testdata[test_index][12],likelihoodAPP)

In [82]:
for data in toBeDetermined:
    label = data[12]
    print(label)
    #outgoingInfo(data)

Mozilla/4.0 (compatible; MSIE 6.0; DynGate)
Mozilla/4.0 (compatible; MSIE 6.0; DynGate)
Mozilla/4.0 (compatible; MSIE 6.0; DynGate)
Mozilla/4.0 (compatible; MSIE 6.0; DynGate)
Mozilla/4.0 (compatible; MSIE 6.0; DynGate)
Mozilla/4.0 (compatible; MSIE 6.0; DynGate)
WinHttpClient
Mozilla/4.0 (compatible; MSIE 6.0; DynGate)
Mozilla/4.0 (compatible; MSIE 6.0; DynGate)
Mozilla/4.0 (compatible; MSIE 6.0; DynGate)
SyncBackPro V7.5.0.0
SyncBackPro V7.5.0.0
Microsoft BITS/7.8
Microsoft BITS/7.8
Microsoft BITS/7.8
Microsoft BITS/7.8
Mozilla/4.0 (compatible; MSIE 6.0; DynGate)
Java/1.8.0_151
Java/1.8.0_151
Java/1.8.0_151
Microsoft-WNS/10.0
Mozilla/4.0 (compatible; MSIE 6.0; DynGate)
Mozilla/4.0 (compatible; MSIE 6.0; DynGate)
Microsoft-CryptoAPI/6.1
Mozilla/4.0 (compatible; MSIE 6.0; DynGate)
WinHttpClient
Mozilla/5.0 (Windows NT 6.1; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/65.0.3325.162 Safari/537.36
Microsoft-CryptoAPI/10.0
Microsoft-CryptoAPI/10.0
Mozilla/4.0 (compatible; MSIE

Mozilla/4.0 (compatible; MSIE 6.0; DynGate)
Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/64.0.3282.186 Safari/537.36
Mozilla/4.0 (compatible; MSIE 6.0; DynGate)
Mozilla/4.0 (compatible; MSIE 6.0; DynGate)
SyncBackPro V7.5.0.0
SyncBackPro V7.5.0.0
Mozilla/4.0 (compatible; MSIE 6.0; DynGate)
Mozilla/4.0 (compatible; MSIE 6.0; DynGate)
Microsoft-WNS/10.0
Mozilla/4.0 (compatible; MSIE 6.0; DynGate)
Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/64.0.3282.186 Safari/537.36
Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/64.0.3282.186 Safari/537.36
APSDaemon.exe (unknown version) CFNetwork/520.32.4
Microsoft-CryptoAPI/10.0
Microsoft-CryptoAPI/10.0
Mozilla/5.0 (Windows NT 6.1; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/65.0.3325.162 Safari/537.36
Mozilla/5.0 (Windows NT 6.1; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/65.0.3325.162 Safari/537

Mozilla/5.0 (Windows NT 6.3; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/65.0.3325.162 Safari/537.36
Mozilla/5.0 (Windows NT 6.3; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/65.0.3325.162 Safari/537.36
Mozilla/5.0 (Windows NT 6.3; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/65.0.3325.162 Safari/537.36
Mozilla/5.0 (Windows NT 6.3; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/65.0.3325.162 Safari/537.36
Mozilla/5.0 (Windows NT 6.3; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/65.0.3325.162 Safari/537.36
Mozilla/5.0 (Windows NT 6.3; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/65.0.3325.162 Safari/537.36
Mozilla/5.0 (Windows NT 6.3; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/65.0.3325.162 Safari/537.36
Mozilla/5.0 (Windows NT 6.3; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/65.0.3325.162 Safari/537.36
Mozilla/5.0 (Windows NT 6.3; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) 

Mozilla/5.0 (Windows NT 6.1; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/65.0.3325.162 Safari/537.36
Mozilla/5.0 (Windows NT 6.1; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/65.0.3325.162 Safari/537.36
Mozilla/5.0 (Windows NT 6.1; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/65.0.3325.162 Safari/537.36
Mozilla/5.0 (X11; Ubuntu; Linux x86_64; rv:58.0) Gecko/20100101 Firefox/58.0
Mozilla/5.0 (Windows NT 6.1; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/65.0.3325.162 Safari/537.36
Mozilla/5.0 (X11; Ubuntu; Linux x86_64; rv:58.0) Gecko/20100101 Firefox/58.0
Mozilla/5.0 (X11; Ubuntu; Linux x86_64; rv:58.0) Gecko/20100101 Firefox/58.0
Mozilla/5.0 (X11; Ubuntu; Linux x86_64; rv:58.0) Gecko/20100101 Firefox/58.0
Mozilla/5.0 (X11; Ubuntu; Linux x86_64; rv:58.0) Gecko/20100101 Firefox/58.0
Mozilla/5.0 (Windows NT 6.1; Win64; x64; rv:52.0) Gecko/20100101 Firefox/52.0 Cyberfox/52.7.0
Mozilla/4.0 (compatible; MSIE 6.0; DynGate)
Mozilla/4.0 (comp

Mozilla/5.0 (Windows NT 6.1; WOW64; Trident/7.0; TBTE; rv:11.0) like Gecko
Mozilla/5.0 (Windows NT 6.1; WOW64; Trident/7.0; TBTE; rv:11.0) like Gecko
Mozilla/5.0 (Windows NT 6.1; WOW64; Trident/7.0; TBTE; rv:11.0) like Gecko
Mozilla/5.0 (Windows NT 6.1; WOW64; Trident/7.0; TBTE; rv:11.0) like Gecko
Mozilla/5.0 (Windows NT 6.1; WOW64; Trident/7.0; TBTE; rv:11.0) like Gecko
Mozilla/5.0 (Windows NT 6.1; WOW64; Trident/7.0; TBTE; rv:11.0) like Gecko
Mozilla/5.0 (Windows NT 6.1; WOW64; Trident/7.0; TBTE; rv:11.0) like Gecko
Mozilla/5.0 (Windows NT 6.1; WOW64; Trident/7.0; TBTE; rv:11.0) like Gecko
Mozilla/5.0 (Windows NT 6.1; WOW64; Trident/7.0; TBTE; rv:11.0) like Gecko
Mozilla/5.0 (Windows NT 6.1; WOW64; Trident/7.0; TBTE; rv:11.0) like Gecko
Mozilla/5.0 (Windows NT 6.1; WOW64; Trident/7.0; TBTE; rv:11.0) like Gecko
Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/65.0.3325.162 Safari/537.36
Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.3

...

Mozilla/5.0 (Windows NT 10.0; WOW64; Trident/7.0; TBTE; rv:11.0) like Gecko
Mozilla/5.0 (Windows NT 10.0; WOW64; Trident/7.0; TBTE; rv:11.0) like Gecko
Mozilla/4.0 (compatible; MSIE 6.0; DynGate)
Mozilla/4.0 (compatible; MSIE 6.0; DynGate)
Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/64.0.3282.186 Safari/537.36
Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/64.0.3282.186 Safari/537.36
Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/64.0.3282.186 Safari/537.36
Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/64.0.3282.186 Safari/537.36
Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/64.0.3282.186 Safari/537.36
Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/64.0.3282.186 Safari/537.36
Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KH

Microsoft-CryptoAPI/6.1
Microsoft-CryptoAPI/6.1
Microsoft-CryptoAPI/6.1
Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/64.0.3282.186 Safari/537.36
Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/64.0.3282.186 Safari/537.36
Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/64.0.3282.186 Safari/537.36
Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/64.0.3282.186 Safari/537.36
Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/64.0.3282.186 Safari/537.36
Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/64.0.3282.186 Safari/537.36
Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/64.0.3282.186 Safari/537.36
Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/64.0.3282.186 Safari/537.36


Mozilla/5.0 (Windows NT 6.1) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/65.0.3325.162 Safari/537.36
Mozilla/5.0 (Windows NT 6.1) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/65.0.3325.162 Safari/537.36
Mozilla/5.0 (Windows NT 6.1) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/65.0.3325.162 Safari/537.36
Mozilla/5.0 (Windows NT 6.1) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/65.0.3325.162 Safari/537.36
Mozilla/5.0 (Windows NT 6.1) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/65.0.3325.162 Safari/537.36
Mozilla/5.0 (Windows NT 6.1) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/65.0.3325.162 Safari/537.36
Mozilla/5.0 (Windows NT 6.1) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/65.0.3325.162 Safari/537.36
Mozilla/5.0 (Windows NT 6.1) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/65.0.3325.162 Safari/537.36
Mozilla/5.0 (Windows NT 6.1) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/65.0.3325.162 Safari/537.36
Mozilla/5.0 (Windows NT 6.1) AppleWebKit/537.36 (KHTML, like Gecko) Chrom

Microsoft-CryptoAPI/6.1
Microsoft-CryptoAPI/6.1
SyncBackPro V7.5.0.0
SyncBackPro V7.5.0.0
Mozilla/5.0 (Windows NT 6.1; WOW64; Trident/7.0; TBTE; rv:11.0) like Gecko
Mozilla/4.0 (compatible; MSIE 6.0; DynGate)
Mozilla/4.0 (compatible; MSIE 6.0; DynGate)
Mozilla/5.0 (Windows NT 6.1; WOW64; Trident/7.0; TBTE; rv:11.0) like Gecko
Mozilla/5.0 (Windows NT 6.1; WOW64; Trident/7.0; TBTE; rv:11.0) like Gecko
Mozilla/5.0 (Windows NT 6.1; WOW64; Trident/7.0; TBTE; rv:11.0) like Gecko
Mozilla/5.0 (Windows NT 6.1; WOW64; Trident/7.0; TBTE; rv:11.0) like Gecko
Mozilla/5.0 (Windows NT 6.1; WOW64; Trident/7.0; TBTE; rv:11.0) like Gecko
Mozilla/5.0 (Windows NT 6.1; WOW64; Trident/7.0; TBTE; rv:11.0) like Gecko
Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/65.0.3325.162 Safari/537.36
Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/65.0.3325.162 Safari/537.36
Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHT

APSDaemon.exe (unknown version) CFNetwork/520.32.4
Mozilla/4.0 (compatible; MSIE 6.0; DynGate)
HCDNClient
Microsoft-CryptoAPI/6.1
Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/64.0.3282.186 Safari/537.36
Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/64.0.3282.186 Safari/537.36
Mozilla/4.0 (compatible; MSIE 6.0; DynGate)
Mozilla/4.0 (compatible; MSIE 6.0; DynGate)
Mozilla/4.0 (compatible; MSIE 6.0; DynGate)
Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/64.0.3282.186 Safari/537.36
Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/64.0.3282.186 Safari/537.36
Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/64.0.3282.186 Safari/537.36
Microsoft-WNS/10.0
Mozilla/4.0 (compatible; MSIE 7.0; Windows NT 5.1) QQBrowser/6.0
Mozilla/4.0 (compatible; MSIE 6.0; DynGate)
Mozilla/5.0 (Windows NT 10.0; 